In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append('..')
from EvGym import config

In [53]:
save = False
df_imb_pre = pd.read_csv("./data/imbalance_prices_2019.csv", parse_dates=["Date"])
df_imb_pre.head()

,Date,PTE,period_from,period_until,upward_incident_reserve,downward_incident_reserve,To regulate up,To regulate down,Incentive component,Consume,Feed,Regulation state
0,2019-01-01,1,00:00,00:15,NaN,NaN,58.01,34.85,0,58.01,34.85,2
1,2019-01-01,2,00:15,00:30,NaN,NaN,NaN,38.85,0,38.85,38.85,-1
2,2019-01-01,3,00:30,00:45,NaN,NaN,NaN,42.85,0,42.85,42.85,-1
3,2019-01-01,4,00:45,01:00,NaN,NaN,NaN,39.03,0,39.03,39.03,-1
4,2019-01-01,5,01:00,01:15,NaN,NaN,NaN,42.85,0,42.85,42.85,-1


In [54]:
df_imb_pre.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35040 entries, 0 to 35039
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Date                       35040 non-null  datetime64[ns]
 1   PTE                        35040 non-null  int64         
 2   period_from                35040 non-null  object        
 3   period_until               35040 non-null  object        
 4   upward_incident_reserve    120 non-null    object        
 5   downward_incident_reserve  1 non-null      object        
 6   To regulate up             19971 non-null  float64       
 7   To regulate down           16910 non-null  float64       
 8   Incentive component        35040 non-null  int64         
 9   Consume                    35040 non-null  float64       
 10  Feed                       35040 non-null  float64       
 11  Regulation state           35040 non-null  int64         
dtypes: d

In [55]:
df_imb_pre["datetime"] = df_imb_pre["Date"] + pd.to_timedelta(df_imb_pre["period_from"]+':00')
df_imb_pre["datehour"] = df_imb_pre["datetime"].dt.strftime("%Y-%m-%d %H:00:00")
df_imb_pre["ts"] = (df_imb_pre["datetime"] - config.starttime_min).dt.total_seconds() / config.timestep
df_imb_pre["price_da"] = 0
df_imb_pre["feed"] = df_imb_pre["Feed"] / 100
df_imb_pre["consume"] = df_imb_pre["Consume"] / 100  

In [56]:
df_imb_pre[["datetime", "datehour", "ts", "Date", "consume", "feed"]].describe()

,datetime,ts,Date,consume,feed
count,35040,35040.000000,35040,35040.000000,35040.000000
mean,2019-07-02 12:27:01.232876800,170940.450342,2019-07-02 00:34:31.232876544,0.425541,0.403911
min,2019-01-01 00:00:00,166560.000000,2019-01-01 00:00:00,-4.876500,-4.876500
25%,2019-04-02 06:56:15,168750.937500,2019-04-02 00:00:00,0.255300,0.244700
50%,2019-07-02 12:52:30,170940.875000,2019-07-02 00:00:00,0.321600,0.311100
75%,2019-10-01 18:48:45,173130.812500,2019-10-01 00:00:00,0.425400,0.409000
max,2019-12-31 23:45:00,175319.750000,2019-12-31 00:00:00,9.361200,9.361200
std,NaN,2528.893561,NaN,0.520039,0.512276


In [63]:
# Make 4 prices
quant = [0, 0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99, 1]
# 1. Feed when minutes == 0
df_imb_a = df_imb_pre[df_imb_pre["datetime"].dt.minute == 0].copy()
df_imb_a = df_imb_a.rename(columns={"feed": "price_im", "datetime": "date"}).reset_index(drop=True)
df_imb_a = df_imb_a[["date", "price_da", "price_im", "ts"]]
print("a----\n", len(df_imb_a), df_imb_a["price_im"].quantile(quant))


# 2. Feed averaged over the hour
df_imb_b = df_imb_pre.groupby(by="datehour").agg(
                                                price_da = ('price_da', 'mean'),
                                                price_im = ('feed', 'mean'),
                                                ts = ('ts', 'min'),
                                                 ).reset_index()

df_imb_b = df_imb_b.rename(columns={"feed": "price_im", "datehour": "date"}).reset_index(drop=True)
df_imb_b = df_imb_b[["date", "price_da", "price_im", "ts"]]
print("b----\n", len(df_imb_b), df_imb_b["price_im"].quantile(quant))

# 3. Consume when minutes == 0
df_imb_c = df_imb_pre[df_imb_pre["datetime"].dt.minute == 0].copy()
df_imb_c = df_imb_c.rename(columns={"consume": "price_im", "datetime": "date"}).reset_index(drop=True)
df_imb_c = df_imb_c[["date", "price_da", "price_im", "ts"]]
print("c----\n", len(df_imb_c), df_imb_c["price_im"].quantile(quant))

# 4. Consume averaged over the hour
df_imb_d = df_imb_pre.groupby(by="datehour").agg(
                                                price_da = ('price_da', 'mean'),
                                                price_im = ('consume', 'mean'),
                                                ts = ('ts', 'min'),
                                                 ).reset_index()

df_imb_d = df_imb_d.rename(columns={"feed": "price_im", "datehour": "date"}).reset_index(drop=True)
df_imb_d = df_imb_d[["date", "price_da", "price_im", "ts"]]
print("d----\n", len(df_imb_d), df_imb_d["price_im"].quantile(quant))
# Columns: date (datetime), price_da (all zeros), price_im, ts
# Also a version without outliers

a----
 8760 0.00   -4.876500
0.01   -1.359117
0.10    0.114480
0.25    0.228275
0.50    0.307700
0.75    0.427000
0.90    1.020790
0.99    2.638292
1.00    7.649200
Name: price_im, dtype: float64
b----
 8759 0.00   -1.949650
0.01   -0.306266
0.10    0.160290
0.25    0.248950
0.50    0.322200
0.75    0.453300
0.90    0.809795
0.99    1.808754
1.00    5.909075
Name: price_im, dtype: float64
c----
 8760 0.00   -4.876500
0.01   -1.227871
0.10    0.158600
0.25    0.258975
0.50    0.334700
0.75    0.469925
0.90    1.161240
0.99    2.680935
1.00    7.649200
Name: price_im, dtype: float64
d----
 8759 0.00   -1.949650
0.01   -0.249123
0.10    0.180980
0.25    0.259838
0.50    0.336150
0.75    0.474900
0.90    0.841190
0.99    1.841059
1.00    5.909075
Name: price_im, dtype: float64


In [68]:
def smooth(x):
    median = x.median()
    iqr = x.quantile(0.75) - x.quantile(0.25)
    y = np.tanh((x - median) / (2*iqr)) * 2*iqr + median
    return y

In [70]:
df_imb_as = df_imb_a.copy()
df_imb_as["price_im"] = smooth(df_imb_as["price_im"])
df_imb_bs = df_imb_b.copy()
df_imb_bs["price_im"] = smooth(df_imb_bs["price_im"])
df_imb_cs = df_imb_c.copy()
df_imb_cs["price_im"] = smooth(df_imb_cs["price_im"])
df_imb_ds = df_imb_d.copy()
df_imb_ds["price_im"] = smooth(df_imb_ds["price_im"])

In [71]:
# Save a, b, c, d, as, bs, cs, ds
if save:
    df_imb_a.to_csv("./data/df_prices_2019_a.csv", index=False)
    df_imb_b.to_csv("./data/df_prices_2019_b.csv", index=False)
    df_imb_c.to_csv("./data/df_prices_2019_c.csv", index=False)
    df_imb_d.to_csv("./data/df_prices_2019_d.csv", index=False)
    df_imb_as.to_csv("./data/df_prices_2019_as.csv", index=False)
    df_imb_bs.to_csv("./data/df_prices_2019_bs.csv", index=False)
    df_imb_cs.to_csv("./data/df_prices_2019_cs.csv", index=False)
    df_imb_ds.to_csv("./data/df_prices_2019_ds.csv", index=False)